In [124]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress
import requests
import hvplot.pandas
import geopandas as gpd

# Import API key
from api_keys import geoapify_key

In [125]:
aqua_csv = pd.read_csv('Resources/AQUASTAT Dissemination System.csv')
mie_csv = pd.read_csv('Resources/ua-mie-1.0.csv')
COW_Country_Codes = pd.read_csv('Resources/COW-country-codes.csv')
micnames = pd.read_csv('Resources/ua-micnames-1.0.csv')

In [126]:
code_to_country = pd.Series(COW_Country_Codes.StateNme.values, index=COW_Country_Codes.CCode).to_dict()

In [127]:
mie_csv['ccode1'] = mie_csv['ccode1'].map(code_to_country)
mie_csv['ccode2'] = mie_csv['ccode2'].map(code_to_country)
mie_csv.head()

micnum  eventnum                    ccode1          ccode2  stmon  stday  \
0       2         1  United States of America  United Kingdom      5     -9   
1       3         1           Austria-Hungary      Yugoslavia     10      7   
2       4         2                   Albania  United Kingdom      5     15   
3       4         3            United Kingdom         Albania     10     22   
4       4         4            United Kingdom         Albania     10     22   

   styear  endmon  endday  endyear  sidea1  action  hostlev  fatalmin1  \
0    1902       5      -9     1902       1       7        3          0   
1    1913      10       7     1913       1       1        2          0   
2    1946       5      15     1946       1      16        4          0   
3    1946      10      22     1946       0       7        3          0   
4    1946      10      22     1946       0       7        3          0   

   fatalmax1  fatalmin2  fatalmax2  version  
0          0          0          0  mie-1.0  
1          0          0          0  mie-1.0  
2          0          0          0  mie-1.0  
3          0          0          0  mie-1.0  
4          0          0          0  mie-1.0

In [128]:
mie_csv = mie_csv.rename(columns={'ccode1': 'Country'})
mie_csv = mie_csv.rename(columns={'ccode2': 'Target Country'})
aqua_csv = aqua_csv.rename(columns={'Area': 'Country'})
COW_Country_Codes = COW_Country_Codes.rename(columns={'StateNme':'Country'})
mie_csv = mie_csv.rename(columns={'styear': 'Year'})



In [129]:
conflict_name = pd.Series(micnames.micname.values, index= micnames.micnum).to_dict()

In [130]:
mie_csv['micnum'] = mie_csv['micnum'].map(conflict_name)
mie_csv.head()

micnum  eventnum  \
0                     Alaska Boundary Dispute (1902)         1   
1  Serbian and Austro-Hungarian Fighting over Alb...         1   
2  British Attempts to Pass the Albanian Corfu Ch...         2   
3  British Attempts to Pass the Albanian Corfu Ch...         3   
4  British Attempts to Pass the Albanian Corfu Ch...         4   

                    Country  Target Country  stmon  stday  Year  endmon  \
0  United States of America  United Kingdom      5     -9  1902       5   
1           Austria-Hungary      Yugoslavia     10      7  1913      10   
2                   Albania  United Kingdom      5     15  1946       5   
3            United Kingdom         Albania     10     22  1946      10   
4            United Kingdom         Albania     10     22  1946      10   

   endday  endyear  sidea1  action  hostlev  fatalmin1  fatalmax1  fatalmin2  \
0      -9     1902       1       7        3          0          0          0   
1       7     1913       1       1        2          0          0          0   
2      15     1946       1      16        4          0          0          0   
3      22     1946       0       7        3          0          0          0   
4      22     1946       0       7        3          0          0          0   

   fatalmax2  version  
0          0  mie-1.0  
1          0  mie-1.0  
2          0  mie-1.0  
3          0  mie-1.0  
4          0  mie-1.0

In [131]:
merged_df = pd.merge(mie_csv, aqua_csv, on=['Country', 'Year'])
merged_df = merged_df.drop(columns=['sidea1','action','stmon','stday','endmon','endday','endyear','version','VariableGroup','Subgroup','fatalmin1','fatalmin2','IsAggregate','Symbol','IsAggregate'])
merged_df.head()

micnum  eventnum   Country  \
0  Thai-Cambodian-Vietnamese Border Conflict and ...        60  Thailand   
1  Thai-Cambodian-Vietnamese Border Conflict and ...        60  Thailand   
2  Thai-Cambodian-Vietnamese Border Conflict and ...        61  Thailand   
3  Thai-Cambodian-Vietnamese Border Conflict and ...        61  Thailand   
4  Thai-Cambodian-Vietnamese Border Conflict and ...        64  Thailand   

  Target Country  Year  hostlev  fatalmax1  fatalmax2  \
0        Vietnam  1982        4          0          0   
1        Vietnam  1982        4          0          0   
2        Vietnam  1982        4          0          1   
3        Vietnam  1982        4          0          1   
4        Vietnam  1983        4          0          0   

                                     Variable      Value    Unit  
0  SDG 6.4.1. Industrial Water Use Efficiency  18.344039  US$/m3  
1    SDG 6.4.1. Services Water Use Efficiency  44.622745  US$/m3  
2  SDG 6.4.1. Industrial Water Use Efficiency  18.344039  US$/m3  
3    SDG 6.4.1. Services Water Use Efficiency  44.622745  US$/m3  
4  SDG 6.4.1. Industrial Water Use Efficiency  19.513973  US$/m3

In [132]:
# Use the Pandas copy function to create DataFrame called geo_merged_df that will gather lat/longs for the Country column
geo_merged_df = merged_df.copy()

# Add an empty columns, 'Lat', 'Lng' to the DataFrame so you can store the lat/longs found using the Geoapify API
geo_merged_df["Lat"] = ""
geo_merged_df["Lng"] = ""
geo_merged_df.head()

micnum  eventnum   Country  \
0  Thai-Cambodian-Vietnamese Border Conflict and ...        60  Thailand   
1  Thai-Cambodian-Vietnamese Border Conflict and ...        60  Thailand   
2  Thai-Cambodian-Vietnamese Border Conflict and ...        61  Thailand   
3  Thai-Cambodian-Vietnamese Border Conflict and ...        61  Thailand   
4  Thai-Cambodian-Vietnamese Border Conflict and ...        64  Thailand   

  Target Country  Year  hostlev  fatalmax1  fatalmax2  \
0        Vietnam  1982        4          0          0   
1        Vietnam  1982        4          0          0   
2        Vietnam  1982        4          0          1   
3        Vietnam  1982        4          0          1   
4        Vietnam  1983        4          0          0   

                                     Variable      Value    Unit Lat Lng  
0  SDG 6.4.1. Industrial Water Use Efficiency  18.344039  US$/m3          
1    SDG 6.4.1. Services Water Use Efficiency  44.622745  US$/m3          
2  SDG 6.4.1. Industrial Water Use Efficiency  18.344039  US$/m3          
3    SDG 6.4.1. Services Water Use Efficiency  44.622745  US$/m3          
4  SDG 6.4.1. Industrial Water Use Efficiency  19.513973  US$/m3

In [133]:
geo_merged_df.dtypes

micnum             object
eventnum            int64
Country            object
Target Country     object
Year                int64
hostlev             int64
fatalmax1           int64
fatalmax2           int64
Variable           object
Value             float64
Unit               object
Lat                object
Lng                object
dtype: object

In [134]:
filtered_df = geo_merged_df[geo_merged_df['Variable'] == 'SDG 6.4.2. Water Stress']
filtered_df.head()

micnum  eventnum      Country Target Country  \
79  The War over Angola of 1975–1976       132       Angola   South Africa   
82                               NaN        47  Afghanistan       Pakistan   
83                               NaN        48  Afghanistan       Pakistan   
84                               NaN        49  Afghanistan       Pakistan   
85                               NaN        50  Afghanistan       Pakistan   

    Year  hostlev  fatalmax1  fatalmax2                 Variable     Value  \
79  1987        4          0          0  SDG 6.4.2. Water Stress   1.27321   
82  1981        3          0          0  SDG 6.4.2. Water Stress  49.69101   
83  1981        4          0          0  SDG 6.4.2. Water Stress  49.69101   
84  1981        3          0          0  SDG 6.4.2. Water Stress  49.69101   
85  1981        3          0          0  SDG 6.4.2. Water Stress  49.69101   

   Unit Lat Lng  
79    %          
82    %          
83    %          
84    %          
85    %

In [135]:
# Base URL for the Geoapify Geocoding API
base_url = "https://api.geoapify.com/v1/geocode/search"

# Create a dictionary to store the results of the API requests
cache = {}

# Iterate over the DataFrame
for index, row in filtered_df.iterrows():
    country = row['Target Country']

    # Check if the result is already in the cache
    if country in cache:
        lat, lng = cache[country]
    else:
        # Set parameters for the API request
        params = {
            'text': country,
            'apiKey': geoapify_key,
        }
        
        # Make the API request
        response = requests.get(base_url, params=params)
        data = response.json()
        
        # Extract latitude and longitude from the response
        try:
            lat = data['features'][0]['properties']['lat']
            lng = data['features'][0]['properties']['lon']
            # Cache the result
            cache[country] = (lat, lng)
        except (KeyError, IndexError):
            # Handle cases where the API doesn't return a valid result
            print(f"Could not find coordinates for {country}")
            lat, lng = None, None

    # Update the DataFrame
    filtered_df.at[index, 'Lat'] = lat
    filtered_df.at[index, 'Lng'] = lng

# Display the updated DataFrame
filtered_df.head()


micnum  eventnum      Country Target Country  \
79  The War over Angola of 1975–1976       132       Angola   South Africa   
82                               NaN        47  Afghanistan       Pakistan   
83                               NaN        48  Afghanistan       Pakistan   
84                               NaN        49  Afghanistan       Pakistan   
85                               NaN        50  Afghanistan       Pakistan   

    Year  hostlev  fatalmax1  fatalmax2                 Variable     Value  \
79  1987        4          0          0  SDG 6.4.2. Water Stress   1.27321   
82  1981        3          0          0  SDG 6.4.2. Water Stress  49.69101   
83  1981        4          0          0  SDG 6.4.2. Water Stress  49.69101   
84  1981        3          0          0  SDG 6.4.2. Water Stress  49.69101   
85  1981        3          0          0  SDG 6.4.2. Water Stress  49.69101   

   Unit        Lat        Lng  
79    % -28.816624  24.991639  
82    %   30.33084  71.247499  
83    %   30.33084  71.247499  
84    %   30.33084  71.247499  
85    %   30.33084  71.247499

In [150]:
total_fatalities = filtered_df.groupby('Target Country')['fatalmax2'].sum().reset_index()
total_fatalities.head()

Target Country  fatalmax2
0    Afghanistan       5629
1        Albania          2
2        Algeria          0
3         Angola        569
4      Argentina          0

In [151]:
# Group by 'Militarized' and count occurrences of each event
event_counts = filtered_df.groupby('Target Country')['micnum'].value_counts().reset_index(name='count')
lat_lng = filtered_df.groupby('Target Country').agg({'Lat': 'first', 'Lng': 'first'}).reset_index()
# Merge the latitude and longitude back into the event_counts DataFrame
event_counts = pd.merge(event_counts, lat_lng, how='left')
# Display the result
event_counts.head()

Target Country                                             micnum  count  \
0    Afghanistan                The Invasion of Afghanistan of 2001     25   
1    Afghanistan  Afghan-Pakistani Border Fighting During the So...     10   
2    Afghanistan  US Bombing of Osama bin Laden, Afghanistan [Op...      1   
3         Angola                       Second Congo War (1998-2002)      8   
4         Angola  Angolan Intervention into Congolese Civil War ...      1   

         Lat        Lng  
0  33.768006  66.238514  
1  33.768006  66.238514  
2  33.768006  66.238514  
3 -11.877577  17.569124  
4 -11.877577  17.569124

In [152]:
# Calculate the average water stress for each target country
average_water_stress = filtered_df.groupby('Target Country')['Value'].mean().reset_index()

# Rename the columns for clarity
average_water_stress.columns = ['Target Country', 'Average Water Stress']

# Print the result
average_water_stress.head()

Target Country  Average Water Stress
0    Afghanistan             88.513709
1        Albania             15.922854
2        Algeria             51.888830
3         Angola              3.151531
4      Argentina              5.734525

In [153]:
print(average_water_stress[average_water_stress['Target Country'] == 'France'])

   Target Country  Average Water Stress
36         France            301.098002


In [141]:
merged_df = pd.merge(event_counts, average_water_stress, on='Target Country', how='inner')
merged_df = merged_df.rename(columns={'count': 'Event Count', 'Value': 'Average Water Stress'})
print(len(merged_df))
merged_df.head()

104


Target Country                                             micnum  \
0    Afghanistan                The Invasion of Afghanistan of 2001   
1    Afghanistan  Afghan-Pakistani Border Fighting During the So...   
2    Afghanistan  US Bombing of Osama bin Laden, Afghanistan [Op...   
3         Angola                       Second Congo War (1998-2002)   
4         Angola  Angolan Intervention into Congolese Civil War ...   

   Event Count        Lat        Lng  Average Water Stress  
0           25  33.768006  66.238514             88.513709  
1           10  33.768006  66.238514             88.513709  
2            1  33.768006  66.238514             88.513709  
3            8 -11.877577  17.569124              3.151531  
4            1 -11.877577  17.569124              3.151531

In [142]:
lat_lng = filtered_df.groupby('Target Country').agg({'Lat': 'first', 'Lng': 'first'}).reset_index()
# Merge the latitude and longitude back into the event_counts DataFrame
total_fatalities = pd.merge(total_fatalities, lat_lng, on='Target Country', how='outer')
# Display the result
total_fatalities.head()


Target Country  fatalmax2        Lat        Lng
0    Afghanistan       5629  33.768006  66.238514
1        Albania          2  41.000028  19.999962
2        Algeria          0  28.000027   2.999983
3         Angola        569 -11.877577  17.569124
4      Argentina          0 -34.996496 -64.967282

In [143]:
merged1_df = pd.merge(total_fatalities, average_water_stress, on='Target Country', how='inner')
merged1_df = merged1_df.rename(columns={'count': 'Event Count', 'Value': 'Average Water Stress'})
merged1_df.head()

Target Country  fatalmax2        Lat        Lng  Average Water Stress
0    Afghanistan       5629  33.768006  66.238514             88.513709
1        Albania          2  41.000028  19.999962             15.922854
2        Algeria          0  28.000027   2.999983             51.888830
3         Angola        569 -11.877577  17.569124              3.151531
4      Argentina          0 -34.996496 -64.967282              5.734525

In [144]:
print(merged1_df[merged1_df['Target Country'] == 'France'])

   Target Country  fatalmax2        Lat       Lng  Average Water Stress
36         France        151  46.603354  1.888334            301.098002


In [145]:
# Load geospatial data
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Merge your data with the geospatial data
# Ensure 'Country' in merged_df matches the country names in the world GeoDataFrame
merged = world.set_index('name').join(merged_df.set_index('Target Country'))
choropleth_map = merged.hvplot.polygons(
    geo=True,  # Enable geographic plotting
    tiles='OSM',  # Add a tile layer for context
    c='Average Water Stress',  # Column to color by
    cmap='OrRd',  # Choose a color map
    line_width=0.5,
    line_color='black',
    width=800,
    height=600,
    title='Water Stress by Country',
    hover_cols=['Target Country', 'Average Water Stress']  # Columns to display on hover
)
choropleth_map

C:\Users\Owner\AppData\Local\Temp\ipykernel_27136\250747982.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]   (Average Water Stress)

In [146]:
# Create a points layer using hvplot
points_layer2 = merged1_df.hvplot.points(
    x = 'Lng',
    y = 'Lat',
    geo = True,
    tiles='OSM',  # Add a tile layer for context
    s ='fatalmax2',  # Size based on Event Count
    color= 'Target Country',  # Color of the points
    alpha=0.6,  # Transparency
    width=800,
    height=600,
    hover_cols=['Target Country', 'fatamax2', 'micnum'],  # Columns to display on hover
    legend=False  # Disable legend for points
)

# Display map

points_layer2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Target Country,fatalmax2)

In [147]:
combined_map2 = choropleth_map * points_layer2
combined_map2

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]   (Average Water Stress)
   .WMTS.II    :WMTS   [Longitude,Latitude]
   .Points.I   :Points   [Lng,Lat]   (Target Country,fatalmax2)